In [1]:
import json
from PIL import Image
import torch
from pycocotools import mask as coco_mask
from pycocotools.coco import COCO
import os
import numpy as np
class myDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, ann_path, transforms=None):
        self.img_dir = img_dir
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir( img_dir )))
        with open(ann_path,'r') as f:
            self.annotations = json.load(f)
         
    def __len__(self):
        return len(self.annotations["images"])

    def __getitem__(self, index):
        img_dir = self.img_dir
        annotations = self.annotations
        transforms = self.transforms

        file_name = annotations["images"][index]['file_name']
        img_path = os.path.join(img_dir, file_name)

        img = Image.open(img_path).convert("RGB")
        height = annotations["images"][index]['height']
        width = annotations["images"][index]['width']
        annotations =[(ann) for ann in self.annotations["annotations"] if ann['image_id'] == index]
        target = {}
        area = []
        labels = []
        masks = []
        boxes = []
        for annotation in annotations:
 
            area.append(annotation["area"])
            labels.append(annotation["category_id"])
            xmin = annotation['bbox'][0]
            xmax = annotation['bbox'][0]+annotation['bbox'][2]
            ymin = annotation['bbox'][1]
            ymax = annotation['bbox'][1]+annotation['bbox'][3]
            boxes.append([xmin, ymin, xmax, ymax])
            # segmentation = annotation["segmentation"]

            # rles = coco_mask.frPyObjects(segmentation, height, width)
            # mask = coco_mask.decode(rles)
            # if len(mask.shape) < 3:
            #     mask = mask[..., None]
            # mask = torch.as_tensor(mask, dtype=torch.uint8)
            # mask = mask.any(dim=2)
            # masks.append(mask.numpy())
            
            # pos = np.where(mask.numpy())
            # xmin = np.min(pos[1])
            # xmax = np.max(pos[1])
            # ymin = np.min(pos[0])
            # ymax = np.max(pos[0])
            #boxes.append([xmin, ymin, xmax, ymax])


        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        target["boxes"] = boxes
        target["labels"] = torch.as_tensor(labels)
        #target["masks"] = torch.as_tensor(masks)
        target["image_id"] = torch.tensor([index])
        target["area"] = torch.tensor(area)
        target["iscrowd"] = torch.zeros(len(annotations), dtype=torch.int64)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

In [ ]:
dataset = myDataset('/content/train',"/content/train/_annotations.coco.json")#,get_transform(train=True))
dataset[0]

In [2]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
 
    # replace the classifier with a new one, that has
    # num_classes which is user-defined
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [ ]:
import transforms as T
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

In [ ]:
import utils
dataset = myDataset('/content/train',"/content/train/_annotations.coco.json", get_transform(train=True))
dataset_test= myDataset('/content/test',"/content/test/_annotations.coco.json", get_transform(train=False))

# split the dataset in train and test set
# torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-5])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-5:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 4

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=5,
                                               gamma=0.1)

In [ ]:
from engine import train_one_epoch, evaluate
# let's train it for 10 epochs
num_epochs = 25

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

In [ ]:
torch.save(model, "/content/gdrive/MyDrive/Devoworm/Train/entire_model.pth")

In [ ]:
# pick one image from the test set
img, _ = dataset_test[1]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])
prediction

In [ ]:
indc = [(ind) for ind,obj in enumerate(prediction[0]['scores']) if obj>0.93]
print(indc)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Reading an image in default mode
image = img.mul(255).permute(1, 2, 0).byte().numpy()

# Window name in which image is displayed
window_name = 'Image'

# Blue color in BGR
color = (255, 0, 0)

# Line thickness of 2 px
thickness = 2
# for box in prediction[0]['boxes'][indc].cpu().numpy():
#   start_point = (box[0], box[1])
#   end_point = (box[2], box[3])
  # image = cv2.rectangle(image, start_point, end_point, color, thickness)
# Using cv2.rectangle() method
# Draw a rectangle with blue line borders of thickness of 2 px
#image = cv2.rectangle(image, start_point, end_point, color, thickness)

# Displaying the image
#cv2_imshow(image)
#plt.imshow(image)


fig, ax = plt.subplots(figsize=(15, 15))

# Display the image
ax.imshow(image)

for box in prediction[0]['boxes'][indc].cpu().numpy():
  ax.add_patch(patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], linewidth=2, edgecolor='r', facecolor='none'))

plt.show()